In [1]:
# Distribution ✨ jit ❇ Demo 💪 
%load_ext autoreload
%autoreload 2

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
# from jax.config import config
# config.update('jax_disable_jit', True)
from functools import partial
import numpy as np

import jax
from jax import pmap, grad
from jax import numpy as jnp
from flax.training.train_state import TrainState

from pyfig import Pyfig
from hwat import FermiNet, sample, compute_ke_b, PotentialEnergy
from hwat import PotentialEnergy, sample, compute_ke_b
import wandb

In [10]:

def compute_metrix(d:dict, specific:list|dict=None):
	from flax.core.frozen_dict import FrozenDict
	# specific: dict = {metric: [variable,]}
	def compute_mean_std(_d, k, v):
		try:
			_d[k+r'_\mu'], _d[k+r'_\sigma'] = np.mean(v), np.std(v)
		except:
			print(f'Could not compute mean or std of {k} type {type(v)}')
		return _d
	
	fancy = dict(
		pe		= r'V(X)',    				
		ke		= r'$\nabla^2',    		
		e		= r'E',						
		log_psi	= r'$\log\psi$', 			
		deltar	= r'\delta_\mathrm{r}',	
		x		= r'r_\mathrm{e}',
	)
	_d = {}
	
	for k,v in d.items():
		k = fancy.get(k, k)
		if isinstance(v, FrozenDict):
			v = v.unfreeze()

		v = jax.tree_map(lambda x: np.array(jax.device_get(x)), v)
		v_mean = jax.tree_map(lambda x: x.mean(), v)
		v_std = jax.tree_map(lambda x: x.std(), v)

		print(k, type(v))

		k_mean, k_std = k+r'_\mu', k+r'_\sigma'
		if isinstance(v, dict):
			for (k_sub, v_m_sub), (_, v_s_sub) in zip(v_mean.items(), v_std.items()):
				_d[f'{k}/{k_sub}_mean'] = v_m_sub
				_d[f'{k}/{k_sub}_std'] = v_s_sub
		else:
			_d[k_mean], _d[k_std] = v_mean, v_std

		# if isinstance(v, dict or list):
		# 	_d[k] = compute_metrix(v)
		# compute_mean_std(_d, k, v)
	for k,v in _d.items():
		print(k, type(v))
	return _d


In [ ]:

args = {
    'n_e': 6,
    'n_u': 3,
    'n_b': 256,
}

c = Pyfig(wandb_mode='online')
print('Pyfig ✅')

part_sample = partial(sample, acc_target=c.data.acc_target)

@partial(pmap, in_axes=(0, 0, 0, 0))
def train_step(rng, state, x, deltar):
    x, v_sam = part_sample(rng, state, x, deltar)

    pe = partial(PotentialEnergy(a=c.data.a, a_z=c.data.a_z).apply, {})(x)
    ke = compute_ke_b(state, x)
    e = pe + ke

    def loss(params):
        return (e * state.apply_fn(params, x)).sum()

    v = dict(
        grad = grad(loss)(state.params),
        pe = pe,
        ke=ke,
        e=e,
        deltar=v_sam['deltar'],
        rng=rng,
        x=x,
        acc=v_sam['acc']
    )
    return state, v
print('Train Step ✅')


rng = c.rng_init
x = c.data.init_walker(rng, n_b=4)
print('Walkers ✅')

@partial(jax.pmap, axis_name='b', in_axes=(0, 0))
def create_train_state(rng, x):
    model = c.partial(FermiNet)  
    params = model.init(rng, x) 
    tx=c.opt.tx
    return TrainState.create(
        apply_fn=model.apply, 
        params=params, 
        tx=tx
    )

state = create_train_state(rng, x)
print('Model ✅')

wandb.define_metric("*", step_metric="tr/step")
deltar = jnp.array([[0.02]]).repeat(c.n_device)
print('Training Variables ✅')

for step in range(1, c.n_step+1):
	state, data = train_step(rng, state, x, deltar)
	state.apply_gradients(grads=data['grad'])	
	print(step)
	
	if step % c.log_metric_step:
		metrix = compute_metrix(data)
        try:
            m = {f'tr/{k}': v for k,v in metrix.items()}
            wandb.log({'tr/step': step, **m})	
        except:
            print(m)
            
	if step % c.log_state_step:
		print('not implemented')

print('Go seek: ', c.wandb.run_path)

path exists, leaving alone


Pyfig ✅
Train Step ✅
Walkers ✅
Model ✅
Training Variables ✅
1


TypeError: Object of type FrozenDict is not JSON serializable

In [ ]:
# 💓 

class tr_data:
    x:jnp.ndarray=x
    rng:jnp.ndarray=rng